<a href="https://colab.research.google.com/github/cbeckler/final_project/blob/cb_etl/ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Reading package lists... Done


In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-01-07 21:34:44--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.3’

postgresql-42.2.16. 100%[===================>] 979.38K  1.61MB/s    in 0.6s    

2023-01-07 21:34:45 (1.61 MB/s) - ‘postgresql-42.2.16.jar.3’ saved [1002883/1002883]



In [3]:
# start spark session that can connect to postgre
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("final-project").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import pandas as pd

# import cancer data

cancer_path = "/content/drive/MyDrive/Data/cancer_incidence.csv"

cancer_df = pd.read_csv(cancer_path)

cancer_df.head()

,index,County,FIPS,"Age-Adjusted Incidence Rate(Ê) - cases per 100,000",Lower 95% Confidence Interval,Upper 95% Confidence Interval,Average Annual Count,Recent Trend,Recent 5-Year Trend (ˆ) in Incidence Rates,Lower 95% Confidence Interval.1,Upper 95% Confidence Interval.1
0,0,"US (SEER+NPCR)(1,10)",0,62.4,62.3,62.6,214614,falling,-2.5,-3,-2
1,1,"Autauga County, Alabama(6,10)",1001,74.9,65.1,85.7,43,stable,0.5,-14.9,18.6
2,2,"Baldwin County, Alabama(6,10)",1003,66.9,62.4,71.7,170,stable,3,-10.2,18.3
3,3,"Barbour County, Alabama(6,10)",1005,74.6,61.8,89.4,25,stable,-6.4,-18.3,7.3
4,4,"Bibb County, Alabama(6,10)",1007,86.4,71,104.2,23,stable,-4.5,-31.4,32.9


In [6]:
# get rid of recent trend nulls (stored as *, _, __)

cancer_df = cancer_df.loc[(cancer_df['Recent Trend']=='rising')|(cancer_df['Recent Trend']=='falling')|(cancer_df['Recent Trend']=='stable')]

# drop unusued columns and rename vars

cancer_df = cancer_df[[' FIPS', 'Recent Trend']].copy()

cancer_df = cancer_df.rename(columns={' FIPS':'FIPS', 'Recent Trend':'recent_trend'})

cancer_df['recent_trend'].value_counts()

stable     2429
falling     200
rising       43
Name: recent_trend, dtype: int64

In [7]:
# import pm data

pm_path = "/content/drive/MyDrive/Data/Daily_PM2.5_Concentrations_All_County__2001-2016.csv"

pm_df = pd.read_csv(pm_path)

pm_df.head()

,year,date,statefips,countyfips,PM25_max_pred,PM25_med_pred,PM25_mean_pred,PM25_pop_pred
0,2001,01JAN2001,1,1,10.664367,10.264546,10.137631,10.188703
1,2001,01JAN2001,1,3,9.803209,8.739505,8.743748,8.811486
2,2001,01JAN2001,1,5,12.087599,11.809159,11.812775,11.802062
3,2001,01JAN2001,1,7,8.579425,8.435394,8.458118,8.448871
4,2001,01JAN2001,1,9,14.399446,13.577741,13.300528,13.231461


In [8]:
# import ozone data

oz_path = "/content/drive/MyDrive/Data/Daily_County-Level_Ozone_Concentrations__2001-2016.csv"

oz_df = pd.read_csv(oz_path)

oz_df.head()

,Year,Month,Day,statefips,countyfips,O3_max_pred,O3_med_pred,O3_mean_pred,O3_pop_pred
0,2001,JAN,1,1,1,31.939058,31.691988,31.680859,31.671226
1,2001,JAN,1,1,3,33.646855,33.170271,32.994775,32.935070
2,2001,JAN,1,1,5,34.288917,34.068507,34.077954,34.086631
3,2001,JAN,1,1,7,30.349767,30.036093,29.931756,29.991733
4,2001,JAN,1,1,9,26.472655,25.776595,25.857571,25.872472


In [9]:
# filter pollution data for years matching cancer data
# it was found med_pred > 100 were outlier values
pm_df = pm_df.loc[(pm_df['year']<=2014)&(pm_df['PM25_med_pred']<=100)]

In [10]:
# check years to ensure they match cancer data range
pm_df.year.unique()

array([2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011,
       2012, 2013, 2014])

In [11]:
# create state + county FIPS to match cancer data FIPS

pm_df['FIPS_pm'] = pm_df['statefips'].astype(str) + pm_df['countyfips'].astype(str)

pm_df['FIPS_pm'] = pm_df['FIPS_pm'].astype(int)

pm_df.head()

,year,date,statefips,countyfips,PM25_max_pred,PM25_med_pred,PM25_mean_pred,PM25_pop_pred,FIPS_pm
0,2001,01JAN2001,1,1,10.664367,10.264546,10.137631,10.188703,11
1,2001,01JAN2001,1,3,9.803209,8.739505,8.743748,8.811486,13
2,2001,01JAN2001,1,5,12.087599,11.809159,11.812775,11.802062,15
3,2001,01JAN2001,1,7,8.579425,8.435394,8.458118,8.448871,17
4,2001,01JAN2001,1,9,14.399446,13.577741,13.300528,13.231461,19


In [12]:
# aggregate pollution data by county

pm_agg = pm_df.groupby('FIPS_pm').agg({'PM25_max_pred':'mean', 'PM25_med_pred':'mean', 'PM25_mean_pred':'mean'})

pm_agg.reset_index(inplace=True)

pm_agg.head()

,FIPS_pm,PM25_max_pred,PM25_med_pred,PM25_mean_pred
0,11,12.239594,11.870589,11.787900
1,13,11.024418,9.822870,9.794272
2,15,11.688170,11.099414,11.084238
3,17,12.280141,11.700512,11.721958
4,19,13.498799,12.690525,12.741736


In [13]:
# aggregate pollution data by county and year

pm_year_agg = pm_df.groupby(['FIPS_pm', 'year']).agg({'PM25_max_pred':'mean', 'PM25_med_pred':'mean', 'PM25_mean_pred':'mean'})

pm_year_agg.reset_index(inplace=True)

pm_year_agg.head()

,FIPS_pm,year,PM25_max_pred,PM25_med_pred,PM25_mean_pred
0,11,2001,13.186807,12.834644,12.784293
1,11,2002,12.490668,12.162930,12.068693
2,11,2003,12.869481,12.485005,12.400802
3,11,2004,13.111945,12.716436,12.619426
4,11,2005,13.543276,13.164472,13.071456


In [14]:
# long to wide format

pm_year_agg = pd.melt(pm_year_agg, id_vars=['FIPS_pm', 'year'], value_vars=['PM25_max_pred', 'PM25_med_pred', 'PM25_mean_pred'])

pm_year_agg['col_name'] = pm_year_agg['variable'] + '_' + pm_year_agg['year'].astype(str)

pm_year_agg = pm_year_agg.pivot(index='FIPS_pm', columns='col_name', values='value')

pm_year_agg.reset_index(inplace=True)

pm_year_agg.head()

col_name,FIPS_pm,PM25_max_pred_2001,PM25_max_pred_2002,PM25_max_pred_2003,PM25_max_pred_2004,PM25_max_pred_2005,PM25_max_pred_2006,PM25_max_pred_2007,PM25_max_pred_2008,PM25_max_pred_2009,...,PM25_med_pred_2005,PM25_med_pred_2006,PM25_med_pred_2007,PM25_med_pred_2008,PM25_med_pred_2009,PM25_med_pred_2010,PM25_med_pred_2011,PM25_med_pred_2012,PM25_med_pred_2013,PM25_med_pred_2014
0,11,13.186807,12.490668,12.869481,13.111945,13.543276,13.638252,14.542888,12.507304,10.279905,...,13.164472,13.224270,14.036194,12.022979,10.003785,10.788455,10.808470,10.438640,10.220977,11.282171
1,13,12.520190,11.269680,12.162325,12.081706,12.994364,11.942907,12.256714,10.514962,9.342412,...,11.503305,10.513930,10.622514,9.247900,8.429134,8.885592,8.865912,8.737817,8.437009,9.635918
2,15,12.364383,11.443971,11.531937,12.543327,12.787813,12.915748,14.095731,11.702072,10.480079,...,12.087744,12.229105,13.112620,11.074860,9.945611,10.689897,10.289959,9.641680,9.644445,10.925400
3,17,13.944184,12.781192,13.162910,13.082518,14.310805,14.060887,14.206462,12.319946,9.992209,...,13.484249,13.370442,13.508796,11.717692,9.575728,10.485379,10.607298,9.992089,9.996376,10.747303
4,19,16.032861,14.493547,14.722058,14.622386,15.809701,15.465637,16.052929,13.357497,10.911080,...,14.797208,14.530090,15.028200,12.598568,10.283973,11.372750,11.053671,10.687267,10.306375,10.994724


In [15]:
# Remove years after 2014 to match cancer data
# no ozone outliers were found in the data
oz_df = oz_df[oz_df['Year']<=2014]

In [16]:
# create state + county FIPS to match cancer data FIPS

oz_df['FIPS_oz'] = oz_df['statefips'].astype(str) + oz_df['countyfips'].astype(str)

oz_df['FIPS_oz'] = oz_df['FIPS_oz'].astype(int)

oz_df.head()

,Year,Month,Day,statefips,countyfips,O3_max_pred,O3_med_pred,O3_mean_pred,O3_pop_pred,FIPS_oz
0,2001,JAN,1,1,1,31.939058,31.691988,31.680859,31.671226,11
1,2001,JAN,1,1,3,33.646855,33.170271,32.994775,32.935070,13
2,2001,JAN,1,1,5,34.288917,34.068507,34.077954,34.086631,15
3,2001,JAN,1,1,7,30.349767,30.036093,29.931756,29.991733,17
4,2001,JAN,1,1,9,26.472655,25.776595,25.857571,25.872472,19


In [17]:
#aggreagate ozone data by county

oz_agg = oz_df.groupby(['FIPS_oz']).agg({'O3_max_pred':'mean','O3_med_pred':'mean','O3_mean_pred':'mean'})

oz_agg.reset_index(inplace=True)

oz_agg.head()

,FIPS_oz,O3_max_pred,O3_med_pred,O3_mean_pred
0,11,41.001571,39.948742,39.921898
1,13,45.070022,41.241604,41.341833
2,15,41.139416,40.185889,40.162437
3,17,40.505549,39.399455,39.389288
4,19,42.714107,41.096201,41.085945


In [18]:
# aggregate ozone data by county and year

oz_year_agg = oz_df.groupby(['FIPS_oz', 'Year']).agg({'O3_max_pred':'mean','O3_med_pred':'mean','O3_mean_pred':'mean'})

oz_year_agg.reset_index(inplace=True)

oz_year_agg.head()

,FIPS_oz,Year,O3_max_pred,O3_med_pred,O3_mean_pred
0,11,2001,42.305986,41.322810,41.306320
1,11,2002,41.864027,40.669792,40.647115
2,11,2003,40.068504,38.835014,38.825136
3,11,2004,39.985884,38.801989,38.819420
4,11,2005,41.686453,40.529218,40.518377


In [19]:
# long to wide format

oz_year_agg = pd.melt(oz_year_agg, id_vars=['FIPS_oz', 'Year'], value_vars=['O3_max_pred', 'O3_med_pred', 'O3_mean_pred'])

oz_year_agg['col_name'] = oz_year_agg['variable'] + '_' + oz_year_agg['Year'].astype(str)

oz_year_agg = oz_year_agg.pivot(index='FIPS_oz', columns='col_name', values='value')

oz_year_agg.reset_index(inplace=True)

oz_year_agg.head()

col_name,FIPS_oz,O3_max_pred_2001,O3_max_pred_2002,O3_max_pred_2003,O3_max_pred_2004,O3_max_pred_2005,O3_max_pred_2006,O3_max_pred_2007,O3_max_pred_2008,O3_max_pred_2009,...,O3_med_pred_2005,O3_med_pred_2006,O3_med_pred_2007,O3_med_pred_2008,O3_med_pred_2009,O3_med_pred_2010,O3_med_pred_2011,O3_med_pred_2012,O3_med_pred_2013,O3_med_pred_2014
0,11,42.305986,41.864027,40.068504,39.985884,41.686453,44.962250,45.038095,41.422075,37.122685,...,40.529218,43.681686,43.838816,40.486148,36.221797,41.289135,40.862455,39.302231,36.364677,37.080065
1,13,46.301171,44.590730,45.091291,45.814931,47.630605,49.781161,47.312026,44.124541,42.082399,...,43.565735,45.922312,43.525804,40.236525,38.505225,42.373774,42.022552,39.985824,37.438556,38.104659
2,15,42.835619,40.764086,39.282389,40.656445,41.673038,44.833797,44.750438,42.070679,38.118018,...,40.585246,43.720973,43.837824,41.187681,37.317680,41.459194,41.776717,39.300123,36.652613,37.449995
3,17,41.606691,41.316442,39.441656,39.402323,42.373669,44.924412,44.941542,40.942346,36.030710,...,41.133554,43.764713,43.850361,39.875838,35.059151,39.827738,39.680734,38.873873,35.961584,36.127933
4,19,43.658069,43.200936,43.007695,41.240142,42.970437,45.503027,46.806477,43.306486,38.762750,...,41.301827,43.677824,45.038788,41.869574,37.462589,42.542686,41.728563,41.646953,37.739042,38.077132


In [20]:
# get row count of initial df

len(cancer_df)

2672

In [21]:
# merge cancer and pollution data

final_df = pd.merge(cancer_df, pm_agg, how='inner', left_on='FIPS', right_on='FIPS_pm')

final_df.head()

,FIPS,recent_trend,FIPS_pm,PM25_max_pred,PM25_med_pred,PM25_mean_pred
0,1101,falling,1101,12.555164,12.013325,11.958884
1,1103,stable,1103,13.287901,12.684610,12.647047
2,1105,stable,1105,11.426605,11.072150,11.079387
3,1107,stable,1107,11.594499,11.146401,11.155309
4,1109,stable,1109,11.352271,10.939546,10.932512


In [22]:
# get row count of matches

len(final_df)

1048

In [23]:
# merge ozone data

final_df = pd.merge(final_df, oz_agg, how='inner', left_on='FIPS', right_on='FIPS_oz')

final_df.head()

,FIPS,recent_trend,FIPS_pm,PM25_max_pred,PM25_med_pred,PM25_mean_pred,FIPS_oz,O3_max_pred,O3_med_pred,O3_mean_pred
0,1101,falling,1101,12.555164,12.013325,11.958884,1101,41.874165,39.939759,39.977164
1,1103,stable,1103,13.287901,12.684610,12.647047,1103,43.308170,41.901690,41.888594
2,1105,stable,1105,11.426605,11.072150,11.079387,1105,39.805057,38.986625,39.022229
3,1107,stable,1107,11.594499,11.146401,11.155309,1107,39.979953,39.173487,39.137133
4,1109,stable,1109,11.352271,10.939546,10.932512,1109,40.690546,39.856677,39.866958


In [24]:
# get count of matches

len(final_df)

1048

In [25]:
# get count of label distribution

final_df['recent_trend'].value_counts()

stable     961
falling     68
rising      19
Name: recent_trend, dtype: int64

In [26]:
# merge year data

final_df = pd.merge(final_df, pm_year_agg, how='left', left_on='FIPS', right_on='FIPS_pm')

final_df = pd.merge(final_df, oz_year_agg, how='left', left_on='FIPS', right_on='FIPS_oz')

final_df.head()

,FIPS,recent_trend,FIPS_pm_x,PM25_max_pred,PM25_med_pred,PM25_mean_pred,FIPS_oz_x,O3_max_pred,O3_med_pred,O3_mean_pred,...,O3_med_pred_2005,O3_med_pred_2006,O3_med_pred_2007,O3_med_pred_2008,O3_med_pred_2009,O3_med_pred_2010,O3_med_pred_2011,O3_med_pred_2012,O3_med_pred_2013,O3_med_pred_2014
0,1101,falling,1101,12.555164,12.013325,11.958884,1101,41.874165,39.939759,39.977164,...,40.567937,43.333084,43.589498,40.750847,36.326936,41.478252,41.055674,39.470118,36.512861,37.457633
1,1103,stable,1103,13.287901,12.684610,12.647047,1103,43.308170,41.901690,41.888594,...,43.226680,44.786660,46.470120,42.158683,38.435499,42.927004,42.164915,41.937580,38.473920,38.021493
2,1105,stable,1105,11.426605,11.072150,11.079387,1105,39.805057,38.986625,39.022229,...,40.612558,43.447473,43.062623,38.970321,34.915892,39.396689,38.999311,37.908802,35.517161,35.654156
3,1107,stable,1107,11.594499,11.146401,11.155309,1107,39.979953,39.173487,39.137133,...,41.574587,43.677365,42.648266,38.709791,35.148054,39.154417,38.971029,38.468993,36.022568,35.750203
4,1109,stable,1109,11.352271,10.939546,10.932512,1109,40.690546,39.856677,39.866958,...,40.589950,43.572161,43.102934,40.463900,36.734288,41.022877,40.996091,38.420984,36.382191,37.127240


In [27]:
# drop dup FIPS columns and return list of all cols

final_df.drop(columns=['FIPS_oz_x', 'FIPS_oz_y', 'FIPS_pm_x', 'FIPS_pm_y'], inplace=True)

final_df.columns

Index(['FIPS', 'recent_trend', 'PM25_max_pred', 'PM25_med_pred',
       'PM25_mean_pred', 'O3_max_pred', 'O3_med_pred', 'O3_mean_pred',
       'PM25_max_pred_2001', 'PM25_max_pred_2002', 'PM25_max_pred_2003',
       'PM25_max_pred_2004', 'PM25_max_pred_2005', 'PM25_max_pred_2006',
       'PM25_max_pred_2007', 'PM25_max_pred_2008', 'PM25_max_pred_2009',
       'PM25_max_pred_2010', 'PM25_max_pred_2011', 'PM25_max_pred_2012',
       'PM25_max_pred_2013', 'PM25_max_pred_2014', 'PM25_mean_pred_2001',
       'PM25_mean_pred_2002', 'PM25_mean_pred_2003', 'PM25_mean_pred_2004',
       'PM25_mean_pred_2005', 'PM25_mean_pred_2006', 'PM25_mean_pred_2007',
       'PM25_mean_pred_2008', 'PM25_mean_pred_2009', 'PM25_mean_pred_2010',
       'PM25_mean_pred_2011', 'PM25_mean_pred_2012', 'PM25_mean_pred_2013',
       'PM25_mean_pred_2014', 'PM25_med_pred_2001', 'PM25_med_pred_2002',
       'PM25_med_pred_2003', 'PM25_med_pred_2004', 'PM25_med_pred_2005',
       'PM25_med_pred_2006', 'PM25_med_pred_200

In [28]:
# get final row count to check for dups

len(final_df)

1048

In [29]:
final_df.head()

,FIPS,recent_trend,PM25_max_pred,PM25_med_pred,PM25_mean_pred,O3_max_pred,O3_med_pred,O3_mean_pred,PM25_max_pred_2001,PM25_max_pred_2002,...,O3_med_pred_2005,O3_med_pred_2006,O3_med_pred_2007,O3_med_pred_2008,O3_med_pred_2009,O3_med_pred_2010,O3_med_pred_2011,O3_med_pred_2012,O3_med_pred_2013,O3_med_pred_2014
0,1101,falling,12.555164,12.013325,11.958884,41.874165,39.939759,39.977164,13.365598,12.650435,...,40.567937,43.333084,43.589498,40.750847,36.326936,41.478252,41.055674,39.470118,36.512861,37.457633
1,1103,stable,13.287901,12.684610,12.647047,43.308170,41.901690,41.888594,15.564124,14.211956,...,43.226680,44.786660,46.470120,42.158683,38.435499,42.927004,42.164915,41.937580,38.473920,38.021493
2,1105,stable,11.426605,11.072150,11.079387,39.805057,38.986625,39.022229,12.768430,11.553522,...,40.612558,43.447473,43.062623,38.970321,34.915892,39.396689,38.999311,37.908802,35.517161,35.654156
3,1107,stable,11.594499,11.146401,11.155309,39.979953,39.173487,39.137133,13.199240,11.938396,...,41.574587,43.677365,42.648266,38.709791,35.148054,39.154417,38.971029,38.468993,36.022568,35.750203
4,1109,stable,11.352271,10.939546,10.932512,40.690546,39.856677,39.866958,12.164619,11.319407,...,40.589950,43.572161,43.102934,40.463900,36.734288,41.022877,40.996091,38.420984,36.382191,37.127240


In [30]:
# create df that includes null rows for pollution and ozone

null_df = pd.merge(cancer_df, pm_agg, how='left', left_on='FIPS', right_on='FIPS_pm')

null_df.head()

,FIPS,recent_trend,FIPS_pm,PM25_max_pred,PM25_med_pred,PM25_mean_pred
0,0,falling,NaN,NaN,NaN,NaN
1,1001,stable,NaN,NaN,NaN,NaN
2,1003,stable,NaN,NaN,NaN,NaN
3,1005,stable,NaN,NaN,NaN,NaN
4,1007,stable,NaN,NaN,NaN,NaN


In [31]:
null_df = pd.merge(null_df, oz_agg, how='left', left_on='FIPS', right_on='FIPS_oz')

null_df.head()

,FIPS,recent_trend,FIPS_pm,PM25_max_pred,PM25_med_pred,PM25_mean_pred,FIPS_oz,O3_max_pred,O3_med_pred,O3_mean_pred
0,0,falling,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1001,stable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1003,stable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1005,stable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1007,stable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
# check that row count matches cancer_df

len(null_df)

2672

In [33]:
# merge in by year data

null_df = pd.merge(null_df, pm_year_agg, how='left', left_on='FIPS', right_on='FIPS_pm')

null_df = pd.merge(null_df, oz_year_agg, how='left', left_on='FIPS', right_on='FIPS_oz')

null_df.head()

,FIPS,recent_trend,FIPS_pm_x,PM25_max_pred,PM25_med_pred,PM25_mean_pred,FIPS_oz_x,O3_max_pred,O3_med_pred,O3_mean_pred,...,O3_med_pred_2005,O3_med_pred_2006,O3_med_pred_2007,O3_med_pred_2008,O3_med_pred_2009,O3_med_pred_2010,O3_med_pred_2011,O3_med_pred_2012,O3_med_pred_2013,O3_med_pred_2014
0,0,falling,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1001,stable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1003,stable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1005,stable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1007,stable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
# drop dup FIPS columns and print list of cols

null_df.drop(columns=['FIPS_oz_x', 'FIPS_oz_y', 'FIPS_pm_x', 'FIPS_pm_y'], inplace=True)

null_df.columns

Index(['FIPS', 'recent_trend', 'PM25_max_pred', 'PM25_med_pred',
       'PM25_mean_pred', 'O3_max_pred', 'O3_med_pred', 'O3_mean_pred',
       'PM25_max_pred_2001', 'PM25_max_pred_2002', 'PM25_max_pred_2003',
       'PM25_max_pred_2004', 'PM25_max_pred_2005', 'PM25_max_pred_2006',
       'PM25_max_pred_2007', 'PM25_max_pred_2008', 'PM25_max_pred_2009',
       'PM25_max_pred_2010', 'PM25_max_pred_2011', 'PM25_max_pred_2012',
       'PM25_max_pred_2013', 'PM25_max_pred_2014', 'PM25_mean_pred_2001',
       'PM25_mean_pred_2002', 'PM25_mean_pred_2003', 'PM25_mean_pred_2004',
       'PM25_mean_pred_2005', 'PM25_mean_pred_2006', 'PM25_mean_pred_2007',
       'PM25_mean_pred_2008', 'PM25_mean_pred_2009', 'PM25_mean_pred_2010',
       'PM25_mean_pred_2011', 'PM25_mean_pred_2012', 'PM25_mean_pred_2013',
       'PM25_mean_pred_2014', 'PM25_med_pred_2001', 'PM25_med_pred_2002',
       'PM25_med_pred_2003', 'PM25_med_pred_2004', 'PM25_med_pred_2005',
       'PM25_med_pred_2006', 'PM25_med_pred_200

In [35]:
# check that row count of df matches cancer_data

len(null_df)

2672

In [36]:
null_df.head()

,FIPS,recent_trend,PM25_max_pred,PM25_med_pred,PM25_mean_pred,O3_max_pred,O3_med_pred,O3_mean_pred,PM25_max_pred_2001,PM25_max_pred_2002,...,O3_med_pred_2005,O3_med_pred_2006,O3_med_pred_2007,O3_med_pred_2008,O3_med_pred_2009,O3_med_pred_2010,O3_med_pred_2011,O3_med_pred_2012,O3_med_pred_2013,O3_med_pred_2014
0,0,falling,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1001,stable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1003,stable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1005,stable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1007,stable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
